In [1]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from enum import Enum



from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    p.patient_code,
    b.first_name,
    b.last_name,
    b.dob,
    dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    dgs.date AS session_date,
    dg.name AS group_name,
    dh.name AS hub_name,
    lc.name AS hub_commune,
    ld.name AS hub_departement,
    dsd.f_komin AS interview_commune,
    ldt.name AS topic,
    au.email as created_by,
    au.username,
    dgs.created_at as date_session_were_entered
FROM
    caris_db.dream_group_attendance dga
        LEFT JOIN
    dream_member dm ON dm.id_patient = dga.id_patient
        LEFT JOIN
    dreams_surveys_data dsd ON dsd.case_id = dm.case_id
        LEFT JOIN
    dream_group_session dgs ON dgs.id = dga.id_group_session
        LEFT JOIN
    dream_group dg ON dg.id = dgs.id_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
        LEFT JOIN
    lookup_dreams_topic ldt ON ldt.id = dgs.topic
        LEFT JOIN
    patient p ON p.id = dga.id_patient
        LEFT JOIN
    beneficiary b ON b.id_patient = dga.id_patient
        left join 
            auth_users au  on au.id=dgs.created_by
WHERE
    dga.value = 'P'
'''

evaluation_performance_attendance = pd.read_sql_query(query, engine, parse_dates=True)


# close the pool of connection
engine.dispose()


True

In [3]:
## get the test excel file from Query (au besoin de verification)
#evaluation_performance_attendance.to_excel('epa.xlsx',index=False,na_rep="NULL")

## Duplication aspect

In [4]:
evaluation_performance_attendance.dtypes

patient_code                         object
first_name                           object
last_name                            object
dob                                  object
interview_date                       object
session_date                         object
group_name                           object
hub_name                             object
hub_commune                          object
hub_departement                      object
interview_commune                    object
topic                                object
created_by                           object
username                             object
date_session_were_entered    datetime64[ns]
dtype: object

In [5]:
epa_deduplicate = evaluation_performance_attendance.drop_duplicates(subset=['patient_code'])

In [6]:
evaluation_performance_attendance.shape
epa_deduplicate.shape

(643107, 15)

(31824, 15)

## Period of control aspect

In [7]:
class Constante(Enum):
    march_1_2021 = pd.to_datetime('2021-10-01')
    specific = pd.to_datetime('2021-11-01')
    #march_1_2021 = "2021-10-01"
    #specific = "2021-11-01"
    workbook = "rapport_agents_saisie_des_sessions_14Octobre2021.xlsx"

In [8]:
#presencesaisie_startmars_endnow = evaluation_performance_attendance[evaluation_performance_attendance["date_session_were_entered"]>=Constante.march_1_2021.value]
#presencesaisie_specific = evaluation_performance_attendance[evaluation_performance_attendance["date_session_were_entered"]>=Constante.specific.value] 
#epa_startmars_endnow = epa_deduplicate[epa_deduplicate['date_session_were_entered']>=Constante.march_1_2021.value]
presencesaisie_startmars_endnow = evaluation_performance_attendance[evaluation_performance_attendance["session_date"]>=Constante.march_1_2021.value]
presencesaisie_specific = evaluation_performance_attendance[evaluation_performance_attendance["session_date"]>=Constante.specific.value] 
epa_startmars_endnow= epa_deduplicate[epa_deduplicate['session_date']>=Constante.march_1_2021.value]

In [9]:
presencesaisie_startmars_endnow.shape
presencesaisie_specific.shape
epa_startmars_endnow.shape

(5714, 15)

(3460, 15)

(291, 15)

## Count by agents in descending order

In [10]:
performance_session_global  = epa_startmars_endnow.groupby(['created_by']).count()
presence_global = presencesaisie_startmars_endnow.groupby(['created_by']).count()
presence_specific = presencesaisie_specific.groupby(['created_by']).count()

In [11]:
performance_session_global.sort_values(ascending=False, by='patient_code',inplace=True)
presence_global.sort_values(ascending=False, by='patient_code',inplace=True)
presence_specific.sort_values(ascending=False, by='patient_code',inplace=True)


In [12]:
performance_session  = pd.DataFrame( {"nombre_de_filles": performance_session_global['patient_code'], f"nombre_de_presence_saisie_depuis_le_{Constante.march_1_2021.value}":presence_global["patient_code"],
                                      f"nombre_de_presence_saisie_au_{Constante.specific.value}":presence_specific["patient_code"]})
performance_session.fillna(0,inplace=True)
performance_session

,nombre_de_filles,nombre_de_presence_saisie_depuis_le_2021-10-01 00:00:00,nombre_de_presence_saisie_au_2021-11-01 00:00:00
created_by,,,
billyjolicoeur95@gmail.com,8.0,144,108.0
blonnichel@yahoo.fr,69.0,912,682.0
byronfabrice9@gmail.com,31.0,512,216.0
diegoarmando.dorlean@carisfoundationintl.org,0.0,1015,1015.0
ernst.belabre@carisfoundationintl.org,4.0,323,0.0
francenelstjean.saed@yahoo.com,142.0,2036,923.0
gerald.poteau@carisfoundationintl.org,1.0,28,0.0
johane.jules@carisfoundationintl.org,1.0,16,0.0
roselord.despeignes@carisfoundationintl.org,21.0,476,476.0


## MODIFY BELOW TO GET THE DIFFERENCE GIVEN LAST WEEK

In [13]:
#week_before_perf = pd.read_excel(Constante.workbook.value, sheet_name="filles_saisies_global", index_col="created_by")
#performance_session["nombre_de_filles_saisie_par_rapport_a_periode_precedente"] =  performance_session["nombre_de_filles"] - week_before_perf["nombre_de_filles"]
performance_session.fillna(0.0,inplace=True)
#performance_session.nombre_de_filles_saisie_par_rapport_a_la_semaine_derniere = performance_session.nombre_de_filles_saisie_par_rapport_a_periode_precedente.astype("int16")
##performance_session.nombre_de_filles_saisie_par_rapport_a_periode_precedente = performance_session.nombre_de_filles_saisie_par_rapport_a_periode_precedente.astype("int16")
performance_session[f"nombre_de_presence_saisie_au_{Constante.specific.value}"] = performance_session[f"nombre_de_presence_saisie_au_{Constante.specific.value}"].astype('int16')

In [14]:
columns_performance_session = performance_session.columns.sort_values().tolist()
performance_session = performance_session.reindex(columns=columns_performance_session)

In [15]:
#performance_session.drop(columns=["nombre_de_presence_saisie_depuis_le_2021-03-01"], inplace=True)
#performance_session.drop(columns=["nombre_de_filles_saisie_par_rapport_a_periode_precedente"], inplace=True)

In [16]:
performance_session

,nombre_de_filles,nombre_de_presence_saisie_au_2021-11-01 00:00:00,nombre_de_presence_saisie_depuis_le_2021-10-01 00:00:00
created_by,,,
billyjolicoeur95@gmail.com,8.0,108,144
blonnichel@yahoo.fr,69.0,682,912
byronfabrice9@gmail.com,31.0,216,512
diegoarmando.dorlean@carisfoundationintl.org,0.0,1015,1015
ernst.belabre@carisfoundationintl.org,4.0,0,323
francenelstjean.saed@yahoo.com,142.0,923,2036
gerald.poteau@carisfoundationintl.org,1.0,0,28
johane.jules@carisfoundationintl.org,1.0,0,16
roselord.despeignes@carisfoundationintl.org,21.0,476,476


## Agent per Commune

In [17]:
performance_agents_par_commune = epa_startmars_endnow.pivot_table(index='created_by',columns='hub_commune',values='patient_code',aggfunc='count',fill_value=0,margins=True, margins_name="Grand Total")
#performance_agents_par_commune.reset_index(inplace=True)

In [18]:
performance_agents_par_commune.sort_values(by=("Grand Total") ,ascending=False,inplace=True)

In [19]:
performance_agents_par_commune

hub_commune,Desdunes,Dessalines,Kenscoff,Liancourt,Port-au-Prince,Pétionville,Saint-Marc,Tabarre,Verrettes,Grand Total
created_by,,,,,,,,,,
Grand Total,18,128,7,25,16,22,44,10,21,291
francenelstjean.saed@yahoo.com,18,124,0,0,0,0,0,0,0,142
blonnichel@yahoo.fr,0,0,0,25,0,0,44,0,0,69
byronfabrice9@gmail.com,0,0,7,0,4,20,0,0,0,31
roselord.despeignes@carisfoundationintl.org,0,0,0,0,0,0,0,0,21,21
twoody103@gmail.com,0,0,0,0,10,0,0,4,0,14
billyjolicoeur95@gmail.com,0,0,0,0,2,0,0,6,0,8
ernst.belabre@carisfoundationintl.org,0,4,0,0,0,0,0,0,0,4
gerald.poteau@carisfoundationintl.org,0,0,0,0,0,1,0,0,0,1


## Reporting Performance de octobre 2021 a date

In [20]:
import openpyxl
pac = pd.ExcelWriter('rapport_agents_saisie_des_sessions.xlsx',engine='openpyxl')
performance_session.to_excel(pac,sheet_name='filles_saisies_global')
performance_agents_par_commune.to_excel(pac,sheet_name="filles_saisies_par_communes")
pac.save()